<a href="https://colab.research.google.com/github/dylanesq/Reliability_II-Assignment/blob/main/Reliability_II_Assignment_1_2nd_part.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import chi2
# from scipy.stats import gumbel_r # Not needed for MLE calculation

# Data
data = pd.DataFrame({
    'i': [1, 2, 3, 4, 5],
    't': [120, 90, 200, 150, 250],
    'delta': [1, 1, 0, 1, 0],
    'x': [100, 110, 90, 95, 85]
})

print("Data:")
print(data)
print()

# Part (a): Likelihood and Log-Likelihood Functions
print("="*60)
print("Part (a): Likelihood and Log-Likelihood Functions")
print("="*60)

def log_likelihood(params, data):
    """
    Log-likelihood for log-location-scale model with Gumbel errors
    y_i = log(t_i) = beta0 + beta1*x_i + b*W_i
    """
    beta0, beta1, b = params
    t = data['t'].values
    x = data['x'].values
    delta = data['delta'].values

    y = np.log(t)

    # Standardized residuals: z_i = W_i = (y_i - eta_i) / b
    z = (y - beta0 - beta1 * x) / b

    # Log-likelihood (Analytical form from derivation: sum [ -delta_i*log(b) - delta_i*z_i - exp(-z_i) ] )
    log_lik_array = -delta * np.log(b) - delta * z - np.exp(-z)

    return np.sum(log_lik_array)

def neg_log_likelihood(params, data):
    # Constraint b > 0
    if params[2] <= 0:
        return np.inf
    return -log_likelihood(params, data)

print("Likelihood function defined:")
print("L(β₀, β₁, b) = Π [ (1/b tᵢ) exp(-zᵢ - e^{-zᵢ}) ]^{δᵢ} [ exp(-e^{-zᵢ}) ]^{1-δᵢ}")
print("Log-likelihood: ℓ = Σ [ -δᵢ log b - δᵢ zᵢ - e^{-zᵢ} ]")
print()

# Part (b): Score Equations
print("="*60)
print("Part (b): Score Equations (Derivatives)")
print("="*60)
print(r"""
Score equations (setting derivatives to zero):
(Where zᵢ = (log(tᵢ) - β₀ - β₁xᵢ) / b)

1.  \frac{\partial \ell}{\partial \beta_0} = -\frac{1}{b} \sum_{i=1}^{n} \left( e^{-z_i} - \delta_i \right) = 0
    \implies \sum_{i=1}^{n} e^{-z_i} = \sum_{i=1}^{n} \delta_i

2.  \frac{\partial \ell}{\partial \beta_1} = -\frac{1}{b} \sum_{i=1}^{n} x_i \left( e^{-z_i} - \delta_i \right) = 0
    \implies \sum_{i=1}^{n} x_i e^{-z_i} = \sum_{i=1}^{n} x_i \delta_i

3.  \frac{\partial \ell}{\partial b} = \frac{1}{b} \sum_{i=1}^{n} \left[ \delta_i (1-z_i) + z_i e^{-z_i} - e^{-z_i} \right] = 0
    \implies \sum_{i=1}^{n} \left[ \delta_i (1-z_i) + e^{-z_i} (z_i - 1) \right] = 0
""")

# Part (c): Maximum Likelihood Estimates
print("="*60)
print("Part (c): Maximum Likelihood Estimates")
print("="*60)

# Initial guess
initial_params = [5, -0.01, 0.5]

# Optimize
result = minimize(neg_log_likelihood, initial_params, args=(data,),
                 method='BFGS', options={'disp': False})

beta0_hat, beta1_hat, b_hat = result.x

print(f"\nMaximum Likelihood Estimates:")
print(f"  β̂₀ = {beta0_hat:.6f}")
print(f"  β̂₁ = {beta1_hat:.6f}")
print(f"  b̂  = {b_hat:.6f}")
print(f"\nLog-likelihood at MLE: {-result.fun:.6f}")
print(f"Optimization successful: {result.success}")

# Part (d): Fisher Information Matrix
print("\n" + "="*60)
print("Part (d): Observed Fisher Information Matrix")
print("="*60)

def compute_hessian(params, data):
    """Compute Hessian matrix numerically"""
    eps = 1e-6
    n_params = len(params)
    hessian = np.zeros((n_params, n_params))

    # Central difference for second derivative (Hessian)
    for i in range(n_params):
        for j in range(n_params):
            if i > j:
                # Use symmetry
                hessian[i, j] = hessian[j, i]
                continue

            p_plus_i = params.copy()
            p_minus_i = params.copy()
            p_plus_i[i] += eps
            p_minus_i[i] -= eps

            if i == j: # Diagonal elements: f''(x) ≈ (f(x+h) - 2f(x) + f(x-h)) / h^2
                hessian[i, i] = (log_likelihood(p_plus_i, data) -
                                 2 * log_likelihood(params, data) +
                                 log_likelihood(p_minus_i, data)) / (eps**2)
            else: # Off-diagonal elements (mixed partial): f_{xy} ≈ (f(x+h, y+h) - f(x+h, y-h) - f(x-h, y+h) + f(x-h, y-h)) / (4h^2)
                p_pp = params.copy(); p_pp[i] += eps; p_pp[j] += eps
                p_pm = params.copy(); p_pm[i] += eps; p_pm[j] -= eps
                p_mp = params.copy(); p_mp[i] -= eps; p_mp[j] += eps
                p_mm = params.copy(); p_mm[i] -= eps; p_mm[j] -= eps

                hessian[i, j] = (log_likelihood(p_pp, data) -
                               log_likelihood(p_pm, data) -
                               log_likelihood(p_mp, data) +
                               log_likelihood(p_mm, data)) / (4 * eps**2)

    return hessian

# Compute observed information: I = -H
params_mle = np.array([beta0_hat, beta1_hat, b_hat])
obs_info = -compute_hessian(params_mle, data)

print("\nObserved Information Matrix I(β̂, b̂):")
print(obs_info)

# Asymptotic Variance-Covariance matrix: V = I⁻¹
var_cov = np.linalg.inv(obs_info)

print("\nAsymptotic Variance-Covariance Matrix V = I⁻¹(β̂, b̂):")
print(var_cov)

print("\nStandard Errors:")
std_errors = np.sqrt(np.diag(var_cov))
print(f"  SE(β̂₀) = {std_errors[0]:.6f}")
print(f"  SE(β̂₁) = {std_errors[1]:.6f}")
print(f"  SE(b̂)  = {std_errors[2]:.6f}")

# Part (e): Likelihood Ratio Test
print("\n" + "="*60)
print("Part (e): Likelihood Ratio Test for H₀: β₁ = 0")
print("="*60)

# Fit reduced model (beta1 = 0)
def log_likelihood_reduced(params, data):
    beta0, b = params
    t = data['t'].values
    delta = data['delta'].values

    y = np.log(t)
    # Reduced model: w_i = (y_i - beta0) / b
    z = (y - beta0) / b

    # Log-likelihood for reduced model (delta_i = 0 in location parameter)
    log_lik_array = -delta * np.log(b) - delta * z - np.exp(-z)

    return np.sum(log_lik_array)

def neg_log_likelihood_reduced(params, data):
    if params[1] <= 0:
        return np.inf
    return -log_likelihood_reduced(params, data)

# Optimize reduced model
initial_params_reduced = [5, 0.5]
result_reduced = minimize(neg_log_likelihood_reduced, initial_params_reduced,
                         args=(data,), method='BFGS', options={'disp': False})

beta0_tilde, b_tilde = result_reduced.x

print(f"\nReduced Model (H₀: β₁ = 0):")
print(f"  β̃₀ = {beta0_tilde:.6f}")
print(f"  b̃  = {b_tilde:.6f}")
ll_reduced = -result_reduced.fun
print(f"  Log-likelihood: {ll_reduced:.6f}")

# Likelihood Ratio Test Statistic
ll_full = -result.fun
lambda_stat = 2 * (ll_full - ll_reduced)

print(f"\nFull Model:")
print(f"  Log-likelihood: {ll_full:.6f}")
print(f"\nLikelihood Ratio Test:")
print(f"  Λ = 2[ℓ(β̂₀, β̂₁, b̂) - ℓ(β̃₀, 0, b̃)] = {lambda_stat:.6f}")

p_value = 1 - chi2.cdf(lambda_stat, df=1)

print(f"\nTest Results:")
print(f"  Test statistic: Λ = {lambda_stat:.6f}")
print(f"  Degrees of freedom: 1")
print(f"  p-value: {p_value:.6f}")
print(f"  Critical value (α=0.05): {chi2.ppf(0.95, 1):.6f}")

if p_value < 0.05:
    print(f"\n  Decision: Reject H₀ at α=0.05 (p-value < 0.05)")
    print(f"  Conclusion: Temperature (β₁) has a significant negative effect on log failure time.")
else:
    print(f"\n  Decision: Fail to reject H₀ at α=0.05")
    print(f"  Conclusion: No significant evidence that temperature affects log failure time.")

Data:
   i    t  delta    x
0  1  120      1  100
1  2   90      1  110
2  3  200      0   90
3  4  150      1   95
4  5  250      0   85

Part (a): Likelihood and Log-Likelihood Functions
Likelihood function defined:
L(β₀, β₁, b) = Π [ (1/b tᵢ) exp(-zᵢ - e^{-zᵢ}) ]^{δᵢ} [ exp(-e^{-zᵢ}) ]^{1-δᵢ}
Log-likelihood: ℓ = Σ [ -δᵢ log b - δᵢ zᵢ - e^{-zᵢ} ]

Part (b): Score Equations (Derivatives)

Score equations (setting derivatives to zero):
(Where zᵢ = (log(tᵢ) - β₀ - β₁xᵢ) / b)

1.  \frac{\partial \ell}{\partial \beta_0} = -\frac{1}{b} \sum_{i=1}^{n} \left( e^{-z_i} - \delta_i \right) = 0
    \implies \sum_{i=1}^{n} e^{-z_i} = \sum_{i=1}^{n} \delta_i 

2.  \frac{\partial \ell}{\partial \beta_1} = -\frac{1}{b} \sum_{i=1}^{n} x_i \left( e^{-z_i} - \delta_i \right) = 0
    \implies \sum_{i=1}^{n} x_i e^{-z_i} = \sum_{i=1}^{n} x_i \delta_i

3.  \frac{\partial \ell}{\partial b} = \frac{1}{b} \sum_{i=1}^{n} \left[ \delta_i (1-z_i) + z_i e^{-z_i} - e^{-z_i} \right] = 0
    \implies \sum_{i=1}^{n}

/usr/local/lib/python3.12/dist-packages/scipy/optimize/_numdiff.py:686: RuntimeWarning: invalid value encountered in subtract
  df = [f_eval - f0 for f_eval in f_evals]
